# It works Jan 6 ---- 
but, every 30 sec is not working and ..
When click the space bar, and image for different folder is still in progress
HELP ME






Also the conf. should be around 80

In [ ]:
# FACES BASIC 1  FACES BASIC 1  FACES BASIC 1 
# REGION OF INTEREST (roi)
# REF VIDEO: https://www.codingforentrepreneurs.com/blog/opencv-python-face-recognition-and-identification

In [2]:
# Import for Capture Photo from video 
import cv2
import os
import shutil
# Import for Augumentation 
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
#import os
import matplotlib.pyplot as plt
# Import all necessary libaries
#import os
import numpy as np
from PIL import Image
#import cv2
import pickle

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
os.chdir('/Users/NidhiAneja/Documents/AI/Cafeteria/OpenCV-Python-Series-master/src/') 
BASE_DIR = '/Users/NidhiAneja/Documents/AI/Cafeteria/OpenCV-Python-Series-master/src/'
base_dir_images = '/Users/NidhiAneja/Documents/AI/Cafeteria/OpenCV-Python-Series-master/src/images/'
# Into the folder of images
image_dir = os.path.join(BASE_DIR, "images")

In [4]:
#--------------------------
#Why does it alawys show directory of ".DS_Store"
#--------------------------
if not os.path.exists('Kunal'):
    os.makedirs('Kunal')
    
#os.listdir()

In [5]:

#---
#BASE_DIR = os.path.dirname(os.path.abspath('src'))
# BASE_DIR
#---
face_cascade = cv2.CascadeClassifier('cascades/data/haarcascade_frontalface_alt2.xml')
#---
y_labels= []
x_train = []
# CREATE DICTIONARY
current_id=0
label_ids= {}

In [7]:
for root, dirs, files in os.walk(image_dir):
    for file in files:
        if file.endswith('png') or file.endswith('jpg'):
            path = os.path.join(root, file)
            label = os.path.basename(root).replace(" ","-").lower()
            #print(label,path)
            #----
            if not label in label_ids:
                label_ids[label] = current_id
                current_id += 1
                
            id_ = label_ids[label]
            #print(label_ids)
                #----
            #y_labels.append(label) # some number
            #x_train.append(path) # verify this image, turn into a NUMPY array, GARY
            # PIL is a image library
            pil_image = Image.open(path).convert("L") # grayscale
            #---
            # RESIZE IMAGES IN TRAINING.....
            #size = (550, 550)
            size = (240, 240)
            final_image = pil_image.resize(size, Image.ANTIALIAS)
            #---
            #image_array = np.array(pil_image, "uint8")
            image_array = np.array(final_image, "uint8")
            #print(image_array)
            # following gives region of interest i.e. faces...
            faces = face_cascade.detectMultiScale(image_array, scaleFactor=1.5, minNeighbors=5)
            
            for (x,y,w,h) in faces:
                roi = image_array[y:y+h, x:x+w]
                x_train.append(roi)
                y_labels.append(id_)
                
#print(y_labels)
#print(x_train)
pickle_path = os.path.join(BASE_DIR, "pickles/face-labels.pickle")
with open(pickle_path, 'wb') as f:
    pickle.dump(label_ids, f)
    

In [11]:
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.train(x_train, np.array(y_labels))
recognizer.save("trainner.yml") 


In [12]:
recognizer.read("trainner.yml") 
labels = {}
with open("labels.pickle", 'rb') as f: # ADD PICKLE TO NAME THE PERSON IN PREDICTION...
    og_labels = pickle.load(f)
    labels = {v:k for k,v in og_labels.items()}    

In [8]:
os.chdir(os.path.join(base_dir_images, 'Kunal'))
os.path.join(BASE_DIR, "images")

In [13]:
# ready to predict....
cap = cv2.VideoCapture(0)
cv2.namedWindow("Who are you?")
cv2.moveWindow("Who are you?", 500, 0)
cv2.resizeWindow("Who are you?", 600,600)
#img_counter = 0
while(True):
    
    # Capture frame-by-frame
    ret, frame = cap.read()
    cv2.imshow("Who are you?", frame)
    gray  = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.5, minNeighbors=5)
    for (x, y, w, h) in faces:
        #print(x,y,w,h)
        roi_gray = gray[y:y+h, x:x+w]
        #print (roi_gray)
        # ADD A FRAME TO THE FACE CAPTURE
        roi_color = frame[y:y+h, x:x+w]        
        # recognize?
        id_, conf = recognizer.predict(roi_gray) # give label id and confidence
        if conf >= 10:
            #print(id_)
            print(labels[id_])
            font = cv2.FONT_HERSHEY_SIMPLEX
            name = labels[id_]
            color = (255,255,255)
            stroke = 2
            cv2.putText(frame, name, (x,y), font,1, color, stroke, cv2.LINE_AA)
        #print(id_)
        print(labels[id_])
        #-------------------------------------------------------
        img_item = "my-image_TEST5.png"
        #cv2.imwrite(img_item, roi_gray)
        cv2.imwrite(img_item, roi_color)
        
        color = (255,0,0) # BGR 0-255
        stroke = 2
        end_cord_x = x + w
        end_cord_y = y + h
        cv2.rectangle(frame, (x,y), (end_cord_x, end_cord_y), color, stroke)
        #cv2.rectangle(img,(384,0),(510,128),(0,255,0),3)
        #------------------------------------------------------
        
        #cv2.imshow('frame',frame)
        if cv2.waitKey(2) & 0xFF == ord('q'):
            break
        k = cv2.waitKey(1)
        '''if k%256 == 32:
            # SPACE pressed
            #img_name = "opencv_frame_{}.png".format(img_counter)
            img_name = "{}.png".format(img_counter)
            cv2.imwrite(img_name, frame)
            print("{} written!".format(img_name))
            img_counter += 1'''
        # Display the resulting frame 
        #cv2.imshow('frame', frame)

kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
peter-dinklage
peter-dinklage
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kuna

KeyboardInterrupt: 

In [14]:

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import os
import numpy as np
from PIL import Image
import pickle

In [ ]:
os.chdir('/Users/NidhiAneja/Documents/AI/Cafeteria/OpenCV-Python-Series-master/src/')
BASE_DIR = os.getcwd()

In [ ]:
image_dir = os.path.join(BASE_DIR, "images")